In [1]:
import random
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import timm
class simCLR(nn.Module):
    def __init__(self, embedding_size, model):
        super(simCLR, self).__init__()
        self.backbone = model
        self.fc = nn.Linear(1024 * 7 * 7, embedding_size)
        torch.nn.init.kaiming_normal_(self.fc.weight)
    def forward(self, x):
        x = self.backbone(x)
        x = x.reshape(-1, 1024 * 7 * 7)
        x = self.fc(x)
        return x
class SRCNN(nn.Module):
    def __init__(self) -> None:
        super(SRCNN, self).__init__()
        # Feature extraction layer.
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, (9, 9), (1, 1), (4, 4)),
            nn.ReLU(True)
        )

        # Non-linear mapping layer.
        self.map = nn.Sequential(
            nn.Conv2d(64, 32, (5, 5), (1, 1), (2, 2)),
            nn.ReLU(True)
        )

        # Rebuild the layer.
        self.reconstruction = nn.Conv2d(32, 3, (5, 5), (1, 1), (2, 2))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)

    # Support torch.script function.
    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        out = self.features(x)
        out = self.map(out)
        out = self.reconstruction(out)

        return out
# srcnn = torch.load("best_psnr.pt", map_location = "cpu")
model_ssl = torch.load("res_ssl_10_.pt", map_location = "cpu")
model_head = timm.create_model("convnext_base.fb_in22k", pretrained = True, num_classes = 25).head

/home/tlrkrwlsmd/.local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
extraction = nn.Sequential(*list(model_ssl.children())[:-2])
# for param in extraction.parameters():
#     param.requires_grad = False

In [3]:
class convnext_base(nn.Module):
    def __init__(self, extraction, head, srcnn):
        super(convnext_base, self).__init__()
        # self.srcnn = srcnn
        self.extraction = extraction
        self.head = head
    
    def forward(self, x):
        # x = self.srcnn(x)
        x = self.extraction(x)
        x = self.head(x)
        return x
model = convnext_base(extraction = extraction, head = model_head, srcnn = None)

In [4]:
CFG = {
    "LEARNING_RATE": 1e-4,
    "EPOCHS": 30,
    "BATCH_SIZE": 32,
    "DEVICE": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
}

In [5]:
class ImageSet(Dataset):
    def __init__(self, img, transform = None, class_name = None, label = None):
        self.img = img
        self.label = label
        self.transform = transform
        self.class_name = class_name
        
    def __len__(self):
        return len(self.img)
    
    def __getitem__(self, idx):
        image = self.img[idx]
        label = self.label[idx]
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        label = class_name[label]
        return image, label

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize([224, 224])                    
])
data = pd.read_csv("train_.csv")
trainset, valset, _, _ = train_test_split(data, data["label"], test_size = 0.2, stratify = data["label"], random_state = 0)
_, augmentset, _, _ = train_test_split(trainset, trainset["label"], test_size = 0.5, stratify = trainset["label"], random_state = 0)
trainset = trainset.reset_index()
trainset.drop(["index"], axis = 1, inplace = True)
valset = valset.reset_index()
valset.drop(["index"], axis = 1, inplace = True)
augmentset = augmentset.reset_index()
augmentset.drop(["index", "Unnamed: 0"], axis = 1, inplace = True)

In [7]:
classes = np.unique(data["label"])
class_name = {name: i for i, name in enumerate(classes)}

In [8]:
train_set = ImageSet(img = trainset["img_path"], transform = transform, class_name = class_name, label = trainset["label"])
validset = ImageSet(img = valset["img_path"], transform = transform, class_name = class_name, label = valset["label"])
# augment_set = ImageSet(img = augmentset["upscale_img_path"], transform = transform, class_name = class_name, label = augmentset["label"])
# trainset = train_set + augment_set

In [9]:
len(trainset)

12667

In [10]:
trainloader = DataLoader(train_set, batch_size = CFG["BATCH_SIZE"], shuffle = True, num_workers = 0)
validloader = DataLoader(validset, batch_size = CFG["BATCH_SIZE"], shuffle = False, num_workers = 0)

In [11]:
# for i, j in trainloader:
#     print(i.size())
#     print(j.size())
#     print(model(i).size())
#     break

In [12]:
optimizer = optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,\
    threshold_mode='abs', min_lr=1e-8, verbose=True)

def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            # validation score를 기준으로 scheduler를 조정한다
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}], Best Val F1 Score : [{_val_score:.5f}]')
    
    return best_model

In [13]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
    
    return _val_loss, _val_score

In [ ]:
infer_model = train(model, optimizer, trainloader, validloader,\
    scheduler, device = CFG["DEVICE"])
torch.save(infer_model, "best_model_CONVNEXT_simclr.pt")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:14<00:00,  6.66it/s]


Epoch [1], Train Loss : [0.53563] Val Loss : [0.19516] Val F1 Score : [0.94330]
Epoch [1], Train Loss : [0.53563], Best Val F1 Score : [0.94330]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:14<00:00,  6.85it/s]


Epoch [2], Train Loss : [0.08084] Val Loss : [0.17839] Val F1 Score : [0.94869]
Epoch [2], Train Loss : [0.08084], Best Val F1 Score : [0.94869]


  5%|████▎                                                                                         | 18/396 [00:10<03:25,  1.84it/s]

In [ ]:
class TestSet(Dataset):
    def __init__(self, img, transform = None):
        self.img = img
        self.transform = transform
        
    def __len__(self):
        return len(self.img)
    
    def __getitem__(self, idx):
        image = self.img[idx]
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image
transform_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize([224, 224])                    
])
test = pd.read_csv("test.csv")
test_set = TestSet(img = test["img_path"], transform = transform_)
test_loader = DataLoader(test_set, batch_size = 1, shuffle = False)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            pred = model(imgs)
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    return preds


In [ ]:
preds = inference(model, test_loader, device = CFG["DEVICE"])
classes = list(class_name.keys())
final = []
for pred in preds:
    final.append(classes[pred])
submit = pd.read_csv("./sample_submission.csv")
submit["label"] = final
submit.to_csv("./submit_30epochs_large.csv", index = False)